In [ ]:


from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os


class QuaternionDataset(Dataset):
    def __init__(self, image_folder, label_folder, transform=None):
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.image_files = sorted(os.listdir(image_folder))
        self.label_files = sorted(os.listdir(label_folder))
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):

        img_path = os.path.join(self.image_folder, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")  # Ensure 3 channels


        lbl_path = os.path.join(self.label_folder, self.label_files[idx])
        with open(lbl_path, 'r') as file:
            label = torch.tensor([float(x) for x in file.readline().strip().split()], dtype=torch.float32)


        if self.transform:
            image = self.transform(image)

        return image, label


image_folder = "/content/drive/MyDrive/Shuttle_main_cropped/images"
label_folder = "/content/drive/MyDrive/Shuttle_main_cropped/quaternion_labels"


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


dataset = QuaternionDataset(image_folder, label_folder, transform=transform)





In [5]:
from torch.utils.data import DataLoader


batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)


for images, labels in dataloader:
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")
    break


Image batch shape: torch.Size([32, 3, 224, 224])
Label batch shape: torch.Size([32, 4])


In [13]:
import torch.nn as nn
import torchvision.models as models


class QuaternionModel(nn.Module):
    def __init__(self):
        super(QuaternionModel, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 4)
        )

    def forward(self, x):
        return self.base_model(x)


model = QuaternionModel()



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
import torch.optim as optim


criterion = nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)


ValueError: optimizer got an empty parameter list

In [20]:
for param in model.base_model.parameters():
    param.requires_grad = False

for param in model.base_model.fc.parameters():
    param.requires_grad = True

for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

base_model.conv1.weight: requires_grad=False
base_model.bn1.weight: requires_grad=False
base_model.bn1.bias: requires_grad=False
base_model.layer1.0.conv1.weight: requires_grad=False
base_model.layer1.0.bn1.weight: requires_grad=False
base_model.layer1.0.bn1.bias: requires_grad=False
base_model.layer1.0.conv2.weight: requires_grad=False
base_model.layer1.0.bn2.weight: requires_grad=False
base_model.layer1.0.bn2.bias: requires_grad=False
base_model.layer1.0.conv3.weight: requires_grad=False
base_model.layer1.0.bn3.weight: requires_grad=False
base_model.layer1.0.bn3.bias: requires_grad=False
base_model.layer1.0.downsample.0.weight: requires_grad=False
base_model.layer1.0.downsample.1.weight: requires_grad=False
base_model.layer1.0.downsample.1.bias: requires_grad=False
base_model.layer1.1.conv1.weight: requires_grad=False
base_model.layer1.1.bn1.weight: requires_grad=False
base_model.layer1.1.bn1.bias: requires_grad=False
base_model.layer1.1.conv2.weight: requires_grad=False
base_model.l

In [21]:
from tqdm import tqdm



num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0


    for batch_idx, (images, labels) in enumerate(tqdm(dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}', dynamic_ncols=True)):

        images, labels = images.to(device), labels.to(device)


        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()


        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}")


Epoch 1/10: 100%|██████████| 626/626 [16:10<00:00,  1.55s/it]


Epoch [1/10], Loss: 0.0509


Epoch 2/10: 100%|██████████| 626/626 [01:26<00:00,  7.24it/s]


Epoch [2/10], Loss: 0.0311


Epoch 3/10: 100%|██████████| 626/626 [01:26<00:00,  7.27it/s]


...

Epoch [8/10], Loss: 0.0178


Epoch 9/10: 100%|██████████| 626/626 [01:24<00:00,  7.41it/s]


Epoch [9/10], Loss: 0.0176


Epoch 10/10: 100%|██████████| 626/626 [01:24<00:00,  7.38it/s]

Epoch [10/10], Loss: 0.0167


In [23]:
from PIL import Image
from torchvision import transforms
import torch


test_image_path = "/content/sequence.0.png"


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


image = Image.open(test_image_path).convert("RGB")
image = transform(image)
image = image.unsqueeze(0)


In [24]:

model = model.to(device)
image = image.to(device)


In [25]:

model.eval()


with torch.no_grad():
    output = model(image)


print(f"Predicted quaternion: {output.squeeze().tolist()}")


Predicted quaternion: [-0.28884392976760864, -0.2874419689178467, 0.13603827357292175, 0.8917813301086426]


In [27]:
for param in model.base_model.parameters():
    param.requires_grad = True

for name, param in model.base_model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [28]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

In [29]:
from tqdm import tqdm

num_fine_tune_epochs = 5
for epoch in range(num_fine_tune_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(dataloader, desc=f"Fine-Tune Epoch {epoch+1}/{num_fine_tune_epochs}"):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item()

    print(f"Fine-Tune Epoch [{epoch+1}/{num_fine_tune_epochs}], Loss: {running_loss / len(dataloader):.4f}")

torch.save(model.state_dict(), "/content/drive/MyDrive/fine_tuned_quaternion_model.pth")
print("Fine-tuned model saved successfully!")

Fine-Tune Epoch 1/5: 100%|██████████| 626/626 [01:24<00:00,  7.40it/s]


Fine-Tune Epoch [1/5], Loss: 0.0090


Fine-Tune Epoch 2/5: 100%|██████████| 626/626 [01:24<00:00,  7.38it/s]


Fine-Tune Epoch [2/5], Loss: 0.0033


Fine-Tune Epoch 3/5: 100%|██████████| 626/626 [01:24<00:00,  7.40it/s]


Fine-Tune Epoch [3/5], Loss: 0.0025


Fine-Tune Epoch 4/5: 100%|██████████| 626/626 [01:24<00:00,  7.44it/s]


Fine-Tune Epoch [4/5], Loss: 0.0014


Fine-Tune Epoch 5/5: 100%|██████████| 626/626 [01:24<00:00,  7.37it/s]


Fine-Tune Epoch [5/5], Loss: 0.0013
Fine-tuned model saved successfully!


In [30]:
model = model.to(device)
image = image.to(device)

In [31]:
model.eval()


with torch.no_grad():
    output = model(image)


print(f"Predicted quaternion: {output.squeeze().tolist()}")

Predicted quaternion: [-0.3518476188182831, -0.20073741674423218, 0.08255627751350403, 0.8791753053665161]


In [38]:
import torch

def quaternion_to_euler(q):
    """
    Convert a quaternion to Euler angles (roll, pitch, yaw).
    Args:
        q: Tensor of shape (batch_size, 4), where each row is (w, x, y, z).
    Returns:
        euler_angles: Tensor of shape (batch_size, 3), where each row is (roll, pitch, yaw).
    """
    q = q / torch.linalg.norm(q, dim=1, keepdim=True)

    w, x, y, z = q[:, 0], q[:, 1], q[:, 2], q[:, 3]

    roll = torch.atan2(2 * (y * w + x * z), 1 - 2 * (y**2 + z**2))
    pitch = torch.asin(2 * (x * y - z * w))
    yaw = torch.atan2(2 * (x * w + y * z), 1 - 2 * (x**2 + y**2))

    euler_angles = torch.stack((roll, pitch, yaw), dim=1)
    return euler_angles


In [39]:
with torch.no_grad():
    output = model(image)


print(f"Predicted quaternion: {quaternion_to_euler(output)}")

Predicted quaternion: tensor([[-2.5529,  0.6692,  0.3251]], device='cuda:0')


In [42]:

quaternion = torch.tensor([-0.286032081, -0.196760952, 0.06013644, 0.935870945])
quaternion = quaternion.unsqueeze(0)

In [43]:
quaternion_to_euler(quaternion)

tensor([[-2.6538,  0.5372,  0.2412]])